# Voice Data Detects Disease

In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

#wfdb is the package that can read WFDB format file
import wfdb

We need to read the information text files first associated with all 208 subjects

In [48]:
# Function to read and process the information text files
def read_info_file(file_path):
  data = {}
  with open(file_path, 'r') as file:
    for line in file:
      # Split the line into key and value parts
      if ':' in line:
        parts = line.split(':')
        if len(parts) >= 2:  # Check if there are at least two parts
          key = parts[0].strip()
          value = parts[1].strip()
          if value == 'NU':  # Handle the 'Not Used' values
            value = np.nan
          elif value.replace(',', '.').replace(' gr', '').replace(' litres', '').isdigit():
          # Convert numbers with commas as decimal points and remove units
            value = float(value.replace(',', '.').replace(' gr', '').replace(' litres', ''))
          elif value.isdigit():  # Convert digit strings to integers
            value = int(value)
            data[key] = value
          elif '\t' in line:
            parts = line.strip().split('\t')
            if len(parts) >= 2:  # Check if there are at least two parts
              key = parts[0].strip()
              value = parts[1].strip()
              data[key] = value
                
  # Convert the dictionary into a DataFrame with a single row
  info_df = pd.DataFrame([data])
    
  return info_df


Using wfdb we will read the .hea files associated with all 208 subjects.

In [49]:
# Function to read the .dat and .hea files
def read_signal_files(dat_file_path, hea_file_path):
    # Use wfdb to read the files and extract features
    pass

Implement a loop to start reading all the files for the 208 subjects within the databse

In [50]:
# Base path to the voice-database folder
base_path = 'voice-database'

# Initialize a list to hold all combined subject data
all_subject_data = []

# Loop over all subject IDs
for subject_id in range(1, 209):  # Assuming IDs from 001 to 208
    # Construct file paths
    info_path = os.path.join(base_path, f'voice{subject_id:03}-info.txt')
    dat_path = os.path.join(base_path, f'voice{subject_id:03}.dat')
    hea_path = os.path.join(base_path, f'voice{subject_id:03}.hea')
    txt_path = os.path.join(base_path, f'voice{subject_id:03}.txt')
    
    print(f"Info Path: {info_path}")
    print(f"Dat Path: {dat_path}")
    print(f"Hea Path: {hea_path}")
    print(f"Txt Path: {txt_path}")
    
    # Read and process info file
    info_data = read_info_file(info_path)
    
    # Read and process signal files (.dat and .hea)
    signal_data = read_signal_files(dat_path, hea_path)
    
    # Combine the info and signal data into one record
    combined_data = ...  # You need to decide how to combine these
    
    # Append the combined data to the list
    all_subject_data.append(combined_data)

# Optionally, convert the list to a DataFrame
all_subjects_df = pd.DataFrame(all_subject_data)


Info Path: voice-database/voice001-info.txt


NetFileNotFoundError: 404 Error: Not Found for url: https://physionet.org/content/voice-database/